# Industry energy demand

### Import necessary modules

In [1]:
# Check if we are running the notebook directly, if so move workspace to parent dir
import sys
import os
currentdir = os.path.abspath(os.getcwd())
if os.path.basename(currentdir) != 'DemandMappingZambia':  
  sys.path.insert(0, os.path.dirname(currentdir))
  os.chdir('..')
  print(f'Move to {os.getcwd()}')

Move to C:\Users\amillot\PycharmProjects\DemandMappingZambia


In [2]:
### Activate geospatial_env first

# Numeric
import numpy as np
import pandas as pd
import math

# System
import os
import shutil
from IPython.display import display, Markdown, HTML, FileLink, FileLinks

# Spatial
import geopandas as gpd
import json
import pyproj
from shapely.geometry import Point, Polygon, MultiPoint
from shapely.wkt import dumps, loads
from shapely.ops import nearest_points
from pyproj import CRS
from osgeo import ogr, gdal, osr
#import fiona


# Mapping / Plotting
from functools import reduce
#import datapane as dp 
#!datapane login --token="9bde41bfbc4ad14119e32086f9f06d2e5db1d5b8"
import folium
from folium.features import GeoJsonTooltip
from folium.plugins import BeautifyIcon
from folium.plugins import HeatMap
import branca.colormap as cm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

In [3]:
import os
import fiona
from IPython.display import display
from rasterstats import zonal_stats
import rasterio
from geojson import Feature, Point, FeatureCollection
import rasterio.fill
from shapely.geometry import shape, mapping
import json
#from earthpy import clip    clip has been deprecated to geopandas
#import earthpy.spatial as es
import tkinter as tk
from tkinter import filedialog, messagebox

import datetime
import warnings
import scipy.spatial
warnings.filterwarnings('ignore')

#import contextily as ctx
import h3 as h3
from shapely.ops import unary_union
from shapely.geometry.polygon import Polygon

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

pd.options.display.float_format = '{:,.4f}'.format

In [4]:
from utils import processing_raster, finalizing_rasters, spatialjoinvectors

### Define directories and dataset names

In [5]:
### Define directories and dataset names
ROOT_DIR = os.path.abspath(os.curdir)
in_path = ROOT_DIR
out_path = ROOT_DIR + "/Outputs/"

## mines layer
ind_data_path = in_path + "/Industry/Data/"
mines_path = ind_data_path + "mines"
# mines_name = 'mines_zambia.gpkg'
mines_name = 'mineral_facilities_zambia_2024'
file_mines = 'Mineral_Facilities_correctedInput.csv'

In [6]:
## Coordinate and projection systems
crs_WGS84 = CRS("EPSG:4326")    # Originan WGS84 coordinate system
crs_proj = CRS("EPSG:32736")    # Projection system for the selected country -- see http://epsg.io/ for more info

In [7]:
ROOT_DIR

'C:\\Users\\amillot\\PycharmProjects\\DemandMappingZambia'

### Define area of interest

In [8]:
try: area
except NameError: area = "COUNTRY"
# area = "Copperbelt"

### Import layers to be used

In [9]:
hexagons = gpd.read_file(out_path + "h3_grid_at_hex.shp")

In [10]:
grid = hexagons

In [11]:
## mines
# mines = gpd.read_file(mines_path + mines_name)

In [12]:
# ## admininstrative boundary
admin_gdf = gpd.read_file(ROOT_DIR + "/admin/" + "gadm41_ZMB.gpkg")


In [13]:
admin_gdf_region = gpd.read_file(ROOT_DIR + "/admin/gadm41_ZMB_shp/" + "gadm41_ZMB_1.shp")

In [14]:
admin_gdf_region

,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,geometry
0,ZMB.1_1,ZMB,Zambia,Central,Broken Hill,NA,Province,Province,NA,ZM.CE,ZM-02,"POLYGON ((27.24544 -15.72238, 27.24094 -15.723..."
1,ZMB.2_1,ZMB,Zambia,Copperbelt,Western,NA,Province,Province,NA,ZM.CO,ZM-08,"POLYGON ((28.64984 -12.83038, 28.65274 -12.833..."
2,ZMB.3_1,ZMB,Zambia,Eastern,Fort Jameson,NA,Province,Province,NA,ZM.EA,ZM-03,"POLYGON ((30.78534 -14.78948, 30.77554 -14.793..."
3,ZMB.4_1,ZMB,Zambia,Luapula,Fort Rosebery,NA,Province,Province,NA,ZM.LP,ZM-04,"POLYGON ((29.57224 -12.19608, 29.56986 -12.196..."
4,ZMB.5_1,ZMB,Zambia,Lusaka,NA,NA,Province,Province,NA,ZM.LS,ZM-09,"POLYGON ((28.90991 -15.9837, 28.90808 -15.9877..."
5,ZMB.6_1,ZMB,Zambia,Muchinga,NA,NA,Province,Province,NA,ZM.MU,ZM-10,"POLYGON ((29.81347 -12.14948, 29.81345 -12.149..."
6,ZMB.7_1,ZMB,Zambia,North-Western,NA,NA,Province,Province,NA,ZM.NW,ZM-06,"POLYGON ((23.53308 -12.9953, 23.9469 -12.99759..."
7,ZMB.8_1,ZMB,Zambia,Northern,NA,NA,Province,Province,NA,ZM.NO,ZM-05,"POLYGON ((29.81334 -11.41218, 29.90984 -11.011..."
8,ZMB.9_1,ZMB,Zambia,Southern,NA,NA,Province,Province,NA,ZM.SO,ZM-07,"POLYGON ((25.89364 -17.97668, 25.88814 -17.975..."
9,ZMB.10_1,ZMB,Zambia,Western,Barotseland,NA,Province,Province,NA,ZM.WE,ZM-01,"POLYGON ((24.95344 -17.54978, 24.94784 -17.551..."


In [15]:
# transmission lines
grid_gdf = gpd.read_file(ROOT_DIR + "/Grid/Zambia - HVLines/" + "HVLines.shp")

In [16]:
grid_gdf

,OBJECTID,ENABLED,CREATIONUS,DATECREATE,DATEMODIFI,LASTUSER,SUBTYPECD,MEASUREDLE,LENGTHSOUR,WORKORDERI,...,circuits,line,designatio,power_line,material,structure,fixme,kv,Voltage_1,geometry
0,1281,1,None,NaT,NaT,None,3,25.0000,None,None,...,10,10,10,10,10,10,10,0.0000,66.0000,"LINESTRING (31.87926 -14.09954, 31.87926 -14.0..."
1,1282,1,None,NaT,NaT,None,3,0.0000,None,None,...,10,10,10,10,10,10,10,0.0000,66.0000,"LINESTRING (31.87928 -14.09958, 31.87928 -14.0..."
2,1283,1,None,NaT,NaT,None,3,"2,958.0000",None,None,...,10,10,10,10,10,10,10,0.0000,66.0000,"LINESTRING (31.87318 -14.07787, 31.87383 -14.0..."
3,1284,1,None,NaT,NaT,None,3,"9,053.0000",None,None,...,10,10,10,10,10,10,10,0.0000,66.0000,"LINESTRING (31.88261 -14.05285, 31.8832 -14.05..."
4,1285,1,None,NaT,NaT,None,3,"2,006.0000",None,None,...,10,10,10,10,10,10,10,0.0000,66.0000,"LINESTRING (31.90315 -13.97378, 31.90356 -13.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,4,2,10,NaT,NaT,10,4,6.0000,10,10,...,None,None,None,None,None,None,None,NaN,NaN,"LINESTRING (25.86684 -12.25216, 25.86747 -12.2..."
927,4,2,10,NaT,NaT,10,4,6.0000,10,10,...,None,None,None,None,None,None,None,NaN,NaN,"LINESTRING (25.86694 -12.252, 25.86753 -12.252..."
928,4,2,10,NaT,NaT,10,4,6.0000,10,10,...,None,None,None,None,None,None,None,NaN,NaN,"LINESTRING (28.75631 -16.51335, 28.75682 -16.5..."
929,4,2,10,NaT,NaT,10,4,6.0000,10,10,...,None,None,None,None,None,None,None,88.0000,92.0000,"LINESTRING (24.67206 -17.27949, 24.67161 -17.2..."


# Part 1. Extract GIS-based attributes

## Extract information from vector layers

Assess the energy consumption per site

In [17]:
## Attribute energy consumption to each copper production site
mines_input_file = pd.read_csv(ind_data_path + file_mines)

In [18]:
## Running the energy calculation function 
from Industry.energy_per_site import *
calc_energy_per_site(mines_path, mines_input_file, mines_name)

Production at a site  Fitwaola Open Pit Mine  in  Zambia  is missing (negative). Value set to zero. Please, change the input in the input file.
Production at a site  Fitwaola Open Pit Mine  in  Zambia  is missing (negative). Value set to zero. Please, change the input in the input file.
Production at a site  Mkushi Heap Leach  in  Zambia  is missing (negative). Value set to zero. Please, change the input in the input file.
Production at a site  Mufulira West Heap-Leach Facility  in  Zambia  is missing (negative). Value set to zero. Please, change the input in the input file.
Production at a site  TD3a, TD3b, TD5, and TD7 Tailings Dams  in  Zambia  is missing (negative). Value set to zero. Please, change the input in the input file.


Extract sum production of mines in each cluster (hex)

In [19]:
## Run the extraction
#grid.drop(['Commodity Production - tonne (tonnes)'], axis=1, inplace=True) ##uncomment if you want to rerun
# columnNameMines = "Commodity Production - tonne (tonnes)"
#columnNameMines = 'Ore processed (tonnes)'
# grid[columnNameMines] = grid[columnNameMines].fillna(0)
# grid.head(4)
# grid[columnNameMines].sum()

columnName_ElecConsumptionMines = 'Energy Elec [TJ]'
columnName_OilConsumptionMines = 'Energy Diesel [TJ]'
file = os.path.join(mines_path, mines_name +".gpkg")
grid, mines = spatialjoinvectors("EnergyMines", columnName_ElecConsumptionMines, admin_gdf, crs_WGS84, grid, "sum", file)

In [20]:
grid[columnName_ElecConsumptionMines].sum()

np.float64(20777.386939748143)

In [21]:
grid.head(4)

,h3_index,n0,n1,n2,n3,n4,n5,index,index_righ,NAME_1,id,geometry,Energy Elec [TJ]
0,8696148afffffff,2803,12112,14180,7459,13483,8748,0,2,Eastern,1,"POLYGON ((32.22444 -12.71369, 32.18722 -12.719...",NaN
1,8696334afffffff,14140,11194,8571,3115,3919,1550,1,5,Muchinga,2,"POLYGON ((31.86748 -12.99196, 31.83018 -12.997...",NaN
2,86961124fffffff,16639,16405,7222,664,8529,9281,2,3,Luapula,3,"POLYGON ((29.69921 -11.07903, 29.66207 -11.084...",NaN
3,86961462fffffff,3843,4261,6699,1872,16124,7533,3,5,Muchinga,4,"POLYGON ((31.99556 -11.97434, 31.95843 -11.979...",NaN


In [22]:
mines.head(4)

,Country,FeatureNam,DsgAttr02,DsgAttr03,DsgAttr06,MemoOther,MemoLoc,Latitude,Longitude,DsgAttr07,...,Metal content [kt],Mine type,Metal processing,Metal process type,Spec energy Elec [GJ/t],Spec energy Diesel [GJ/t],Energy Elec [TJ],Energy Diesel [TJ],id,geometry
0,Zambia,Baluba Center Underground Mine,Copper,Concentrate,2017,Ore. Operations suspended. Facility on care-an...,<null>,-13.0486,28.3381,"1,500,000.0000",...,14.6775,Underground,,,11.7500,2.1500,172.4606,31.5566,1,POINT (28.33806 -13.04861)
1,Zambia,Bwana Mkubwa Solvent Extraction-Electrowinning...,Copper,Metal,2017,Copper cathode.,<null>,-13.0423,28.7094,"52,000.0000",...,21.5280,,Hydrometallurgical,"Solvent Extraction-Electrowinning, Electrowinn...",12.4950,2.2050,268.9924,47.4692,2,POINT (28.70935 -13.0423)
2,Zambia,Chambishi Cobalt Plant,Cobalt,Metal,2017,Cobalt metal.,75 kilometers northwest of Ndola,-12.6500,28.0412,"5,500.0000",...,0.7480,,Smelting/Refining,,13.5700,0.0000,10.1504,0.0000,3,POINT (28.04117 -12.64998)
3,Zambia,Chambishi Facility,Copper,Metal,2017,Copper cathode.,<null>,-12.6500,28.0412,"8,000.0000",...,3.3120,,Smelter+Refinery,,10.5460,3.4380,34.9284,11.3867,4,POINT (28.04117 -12.64998)


# Part 2. Compute demand

In [23]:
# Enegery balance from UN Stats
energyBalance_path = "EnergyBalance/"
file_energyBalance = "UNSD+DF_UNData_EnergyBalance+1.0_Zambia.csv"
eb = pd.read_csv(energyBalance_path + file_energyBalance)
code_elec = "B07_EL"
code_oil = "B03_OP"
code_ind_nFM=  "B29_1214a"
code_ind_mining =  "B33_1214e"

elec_nonFerrousMetals_TJ = eb.loc[(eb['COMMODITY'] == code_elec) & (eb['TRANSACTION'] == code_ind_nFM) & (eb['TIME_PERIOD'] == 2019 ), 'OBS_VALUE'] #TJ
elec_mining_TJ = eb.loc[(eb['COMMODITY'] == code_elec) & (eb['TRANSACTION'] == code_ind_mining) & (eb['TIME_PERIOD'] == 2019 ), 'OBS_VALUE'] #TJ
oil_nonFerrousMetals_TJ = eb.loc[(eb['COMMODITY'] == code_oil) & (eb['TRANSACTION'] == code_ind_nFM) & (eb['TIME_PERIOD'] == 2019 ), 'OBS_VALUE'] #TJ
oil_mining_TJ = eb.loc[(eb['COMMODITY'] == code_oil) & (eb['TRANSACTION'] == code_ind_mining) & (eb['TIME_PERIOD'] == 2019 ), 'OBS_VALUE'] #TJ

elec_nonFerrousMetals_TJ = pd.to_numeric(elec_nonFerrousMetals_TJ.str.replace(',', '')) # convert to numeric and remove commas
elec_nonFerrousMetals_TJ = elec_nonFerrousMetals_TJ.iloc[0]
elec_mining_TJ = pd.to_numeric(elec_mining_TJ.str.replace(',', '')) # convert to numeric and remove commas
elec_mining_TJ = elec_mining_TJ.iloc[0]
elec = elec_nonFerrousMetals_TJ+elec_mining_TJ
oil_nonFerrousMetals_TJ = pd.to_numeric(oil_nonFerrousMetals_TJ.str.replace(',', '')) # convert to numeric and remove commas
oil_nonFerrousMetals_TJ = oil_nonFerrousMetals_TJ.iloc[0]
oil_mining_TJ = pd.to_numeric(oil_mining_TJ.str.replace(',', '')) # convert to numeric and remove commas
oil_mining_TJ = oil_mining_TJ.iloc[0]
oil = oil_nonFerrousMetals_TJ+oil_mining_TJ

energy = elec+oil

print("total statistical electricity consumption of nonFerrousMetals:",f"{elec_nonFerrousMetals_TJ:,.1f}", "TJ")
print("total statistical electricity consumption in mining:",f"{elec_mining_TJ:,.1f}", "TJ")
print("total statistical electricity consumption of nonFerrousMetals and mining:",f"{elec:,.1f}", "TJ")

print("total statistical oil consumption of nonFerrousMetals:",f"{oil_nonFerrousMetals_TJ:,.1f}", "TJ")
print("total statistical oil consumption in mining:",f"{oil_mining_TJ:,.1f}", "TJ")
print("total statistical oil consumption of nonFerrousMetals and mining:",f"{oil:,.1f}", "TJ")

print("total statistical  energy consumption of nonFerrousMetals and mining:",f"{energy:,.1f}", "TJ")

total statistical electricity consumption of nonFerrousMetals: 22,897.0 TJ
total statistical electricity consumption in mining: 818.0 TJ
total statistical electricity consumption of nonFerrousMetals and mining: 23,715.0 TJ
total statistical oil consumption of nonFerrousMetals: 1,191.1 TJ
total statistical oil consumption in mining: 2,689.7 TJ
total statistical oil consumption of nonFerrousMetals and mining: 3,880.8 TJ
total statistical  energy consumption of nonFerrousMetals and mining: 27,595.8 TJ


In [24]:
# Assess total energy consumption and total production
total_elec_consum = mines[columnName_ElecConsumptionMines].sum() # total
total_oil_consum = mines[columnName_OilConsumptionMines].sum() # total
total_energy_consum = total_elec_consum + total_oil_consum
coverage_energy_nonFe_mining = total_energy_consum/energy

# print("total production:", f"{total_production/10**3:,.0f}", "kt")
print("total calculated electricity ", f"{total_elec_consum:,.1f}",", diesel ", f"{total_oil_consum:,.1f}", " and energy consumption:", f"{total_energy_consum:,.1f}", "TJ")
print("Coverage energy in total statistical nonFerrousMetals and mining consumption:", f"{coverage_energy_nonFe_mining*100:,.0f}", "%")

total calculated electricity  20,777.4 , diesel  9,684.3  and energy consumption: 30,461.7 TJ
Coverage energy in total statistical nonFerrousMetals and mining consumption: 110 %


In [25]:
#Allocate to each hexagon the industry energy consumption
#grid["IndEnergy"]=grid[columnNameMines]*energycons_perton
# adjust energy consumption if coverage is the whole country
if area == "COUNTRY":
    grid["IndEne_TJ"]=grid[columnName_ElecConsumptionMines]/coverage_energy_nonFe_mining  # TJ
else:
    grid["IndEne_TJ"]=grid[columnName_ElecConsumptionMines] # TJ
grid.head(3)

,h3_index,n0,n1,n2,n3,n4,n5,index,index_righ,NAME_1,id,geometry,Energy Elec [TJ],IndEne_TJ
0,8696148afffffff,2803,12112,14180,7459,13483,8748,0,2,Eastern,1,"POLYGON ((32.22444 -12.71369, 32.18722 -12.719...",NaN,NaN
1,8696334afffffff,14140,11194,8571,3115,3919,1550,1,5,Muchinga,2,"POLYGON ((31.86748 -12.99196, 31.83018 -12.997...",NaN,NaN
2,86961124fffffff,16639,16405,7222,664,8529,9281,2,3,Luapula,3,"POLYGON ((29.69921 -11.07903, 29.66207 -11.084...",NaN,NaN


In [26]:
total_industryenergy = grid["IndEne_TJ"].sum()
#print("Industry electricity consumption:",f"{total_industryenergy/10**6:,.0f}", "TWh")
print("Industry electricity consumption in",f"{area}", ":",f"{total_industryenergy/3600:,.1f}", "TWh")

Industry electricity consumption in COUNTRY : 5.2 TWh


In [27]:
grid.to_file(out_path + 'ind_energy_map.shp', index=False)
grid.to_file(out_path + f'ind_energy_map.geojson', driver='GeoJSON', index=False)  
grid.head(3)

,h3_index,n0,n1,n2,n3,n4,n5,index,index_righ,NAME_1,id,geometry,Energy Elec [TJ],IndEne_TJ
0,8696148afffffff,2803,12112,14180,7459,13483,8748,0,2,Eastern,1,"POLYGON ((32.22444 -12.71369, 32.18722 -12.719...",NaN,NaN
1,8696334afffffff,14140,11194,8571,3115,3919,1550,1,5,Muchinga,2,"POLYGON ((31.86748 -12.99196, 31.83018 -12.997...",NaN,NaN
2,86961124fffffff,16639,16405,7222,664,8529,9281,2,3,Luapula,3,"POLYGON ((29.69921 -11.07903, 29.66207 -11.084...",NaN,NaN


In [28]:
grid[columnName_ElecConsumptionMines]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
18385   NaN
18386   NaN
18387   NaN
18388   NaN
18389   NaN
Name: Energy Elec [TJ], Length: 18390, dtype: float64